# Fit Cox Models

## Initialize

In [1]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output
import ray
import datetime
import subprocess
import warnings
import lifelines
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
lifelines.__version__

'0.26.4'

In [3]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas_220603_fullrun"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

USER = 'loockl'
BASE = pathlib.Path(f"/home/{USER}/")
EXPERIMENT_NAME = '220603_fullrun'
#TEMPLATE_CONFIG = f"{BASE}/config/"  # template yaml to use
TRAIN_SCRIPT = f"{BASE}/riskiano/riskiano/experiments/lukas/phewas/22_retina_phewas_notebooks/1_processing/08_coxph_fit_partition.py"
ACTIVATE_ENV_CMD = """conda activate retina_risk"""

TAG = '220603_fullrun'
JOBNAME = 'fit_coxph'

name_dict = {
    "predictions_cropratio0.3": "ConvNextSmall(Retina)+MLP_cropratio0.3",
    "predictions_cropratio0.5": "ConvNextSmall(Retina)+MLP_cropratio0.5",
    "predictions_cropratio0.8": "ConvNextSmall(Retina)+MLP_cropratio0.8",
}

partitions = [i for i in range(22)]
partitions

/sc-projects/sc-proj-ukb-cvd


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

## Submit CoxPH Jobs

In [4]:
os.makedirs(f"/home/{USER}/tmp/{EXPERIMENT_NAME}/job_submissions", exist_ok=True)
os.makedirs(f"/home/{USER}/tmp/{EXPERIMENT_NAME}/job_configs", exist_ok=True)
os.makedirs(f"/home/{USER}/tmp/{EXPERIMENT_NAME}/job_outputs", exist_ok=True)

In [64]:
def make_job_script(user, job_name, partition):

    job_script_str = (
        f"""#!/bin/bash

#SBATCH --job-name={job_name}  # Specify job name
#SBATCH --nodes=1              # Specify number of nodes
#SBATCH --mem=500G              # Specify number of nodes
#SBATCH --time=4:00:00        # Set a limit on the total run time
#SBATCH --tasks-per-node=1
#SBATCH --exclusive

{ACTIVATE_ENV_CMD}

ray start --head --port=6378 --num-cpus 16
python {TRAIN_SCRIPT} {partition}"""
            )

    return job_script_str


In [65]:
def submit(path, job_name, job_script, time_stamp=None):
    if not time_stamp:
        time_stamp = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

    script_path_long = f"{path}/{job_name}_{time_stamp}.sh"

    with open(script_path_long, "w") as outfile:
        outfile.write(job_script)
    script_path = f"{path}/{job_name}.sh"
    try:
        os.unlink(script_path)
    except FileNotFoundError:  # because we cannot overwrite symlinks directly
        pass
    os.symlink(os.path.realpath(script_path_long), script_path)

    output_path = f"/home/{USER}/tmp/{EXPERIMENT_NAME}/job_outputs/{job_name}"

    print(job_script)
    print("\n\nSubmission:\n===========\n")
    sub_cmd = (
        f"sbatch --error={output_path}_%j_stderr.out --output={output_path}_%j_stdout.out <"
        f" {script_path}"
    )
    print(sub_cmd)

    ret = subprocess.run(sub_cmd, shell=True, cwd=os.getcwd(), capture_output=True)
    print(ret.stdout.decode())

In [73]:
partitions = [i for i in range(22)]
#partitions = [i for i in range(1)]

In [71]:
import time

jobids = []
for partition in partitions:

    job_script = make_job_script(user=USER, job_name=JOBNAME, partition=partition)

    jobid = submit(
        path=f"/home/{USER}/tmp/{EXPERIMENT_NAME}/job_submissions",
        job_name=JOBNAME + f"_{partition}",
        job_script=job_script,
    )

    jobids.append(jobid)
    #time.sleep(2)

print(jobids)

#!/bin/bash

#SBATCH --job-name=fit_coxph  # Specify job name
#SBATCH --nodes=1              # Specify number of nodes
#SBATCH --mem=500G              # Specify number of nodes
#SBATCH --time=4:00:00        # Set a limit on the total run time
#SBATCH --tasks-per-node=1
#SBATCH --exclusive

conda activate retina_risk

ray start --head --port=6378 --num-cpus 16
python /home/loockl/riskiano/riskiano/experiments/lukas/phewas/22_retina_phewas_notebooks/1_processing/08_coxph_fit_partition.py 20


Submission:

sbatch --error=/home/loockl/tmp/220603_fullrun/job_outputs/fit_coxph_20_%j_stderr.out --output=/home/loockl/tmp/220603_fullrun/job_outputs/fit_coxph_20_%j_stdout.out < /home/loockl/tmp/220603_fullrun/job_submissions/fit_coxph_20.sh
Submitted batch job 159259

#!/bin/bash

#SBATCH --job-name=fit_coxph  # Specify job name
#SBATCH --nodes=1              # Specify number of nodes
#SBATCH --mem=500G              # Specify number of nodes
#SBATCH --time=4:00:00        # Set a limit on the tot

## Check progress

In [72]:
cox_paths = !ls "/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas_220603_fullrun/data/220603_fullrun/coxph/models/"
len(cox_paths), cox_paths[0]
#path_df = pd.DataFrame(data = [p[:-2] for p in cox_paths]).rename(columns={0:"path"})
#path_df[["endpoint_1", "endpoint_2", "score", "partition"]] = path_df.path.str.split("_", expand=True,) 
#path_df["endpoint"] = path_df["endpoint_1"] + "_" + path_df["endpoint_2"] 

#path_df.value_counts(["partition"]).to_frame()

(695574,
 'OMOP_4306655_Age+Sex_ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.3_0.p')

## Fig Crashing CoxPH models

In [17]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas_220603_fullrun"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

experiment = '220603_fullrun'
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

name_dict = {
    "predictions_cropratio0.3": "ConvNextSmall(Retina)+MLP_cropratio0.3",
    "predictions_cropratio0.5": "ConvNextSmall(Retina)+MLP_cropratio0.5",
    "predictions_cropratio0.8": "ConvNextSmall(Retina)+MLP_cropratio0.8",
}

partitions = [i for i in range(22)]
partitions

/sc-projects/sc-proj-ukb-cvd


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [39]:
in_path = pathlib.Path(f"{experiment_path}/coxph/input")
models = [f.name for f in in_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]
models

['ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.3',
 'ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.8',
 'ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.5']

In [23]:
import pandas as pd
endpoints_md = pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_retinal_risk/data/220602/endpoints.csv')
endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_retinal_risk/data/220602/endpoints.csv').endpoint.values)])
endpoints_md

,Unnamed: 0,endpoint,eligable,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf
0,0,OMOP_4306655,61256,3548,0.057921,4306655.00,All-Cause Death,Death,Both,NaN,NaN,NaN
1,1,phecode_002,60945,658,0.010797,2.00,Staphylococcus,ID,Both,0.0,2.0,0.0
2,2,phecode_002-1,61010,486,0.007966,2.10,Staphylococcus aureus,ID,Both,0.0,2.0,1.0
3,3,phecode_003,60757,1017,0.016739,3.00,Escherichia coli,ID,Both,0.0,3.0,1.0
4,4,phecode_004,60584,494,0.008154,4.00,Streptococcus,ID,Both,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1166,1166,phecode_977-52,31669,520,0.016420,977.52,Hormone replacement therapy (postmenopausal),Rx,Female,0.0,977.0,1.0
1167,1167,phecode_977-7,60032,2231,0.037164,977.70,Long term (current) use of insulin or oral hyp...,Rx,Both,0.0,977.0,0.0
1168,1168,phecode_977-71,60936,472,0.007746,977.71,Long term (current) use of insulin,Rx,Both,0.0,977.0,1.0
1169,1169,phecode_977-72,60207,2148,0.035677,977.72,Long term (current) use of oral hypoglycemic d...,Rx,Both,0.0,977.0,1.0


In [20]:
import yaml
import pickle
import zstandard

def get_score_defs():

    with open(r'/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas_220603_fullrun/data/score_definitions.yaml') as file:
        score_defs = yaml.full_load(file)
    
    return score_defs

def get_features(endpoint, score_defs):
    features = {
        model: {
            #"Age+Sex": score_defs["AgeSex"],
            #"Retina": [endpoint],
            #"SCORE2": score_defs["SCORE2"],
            #"ASCVD": score_defs["ASCVD"],
            "QRISK3": score_defs["QRISK3"],
            #"Age+Sex+Retina": score_defs["AgeSex"] + [endpoint],
            #"SCORE2+Retina": score_defs["SCORE2"] + [endpoint],
            #"ASCVD+Retina": score_defs["ASCVD"] + [endpoint],
            "QRISK3+Retina": score_defs["QRISK3"] + [endpoint],
            }
        for model in models}
    return features

def load_pickle(fp):
    with open(fp, "rb") as fh:
        dctx = zstandard.ZstdDecompressor()
        with dctx.stream_reader(fh) as decompressor:
            data = pickle.loads(decompressor.read())
    return data

def fit_cox(data_fit, feature_set, covariates, endpoint, penalizer, step_size=1):
    cph = CoxPHFitter(penalizer=penalizer)
    cph.fit(data_fit, f"{endpoint}_time", f"{endpoint}_event", step_size=step_size)
    return cph

score_defs = get_score_defs()

In [61]:
def clean_covariates(endpoint, covariates):
    if endpoint=="phecode_181": # Autoimmune disease
        covariates = [c for c in covariates if c!="systemic_lupus_erythematosus"]
    if endpoint=="phecode_202": # Diabetes
        covariates = [c for c in covariates if c not in ['diabetes1', 'diabetes2', 'diabetes']]
    if endpoint=="phecode_202-1": # Diabetes 1
        covariates = [c for c in covariates if c!="diabetes1"]
    if endpoint=="phecode_202-2": # Diabetes 1
        covariates = [c for c in covariates if c!="diabetes2"]
    if endpoint=="phecode_286": # Mood [affective] disorders
        covariates = [c for c in covariates if c not in ['bipolar_disorder', 'major_depressive_disorder']]
    if endpoint=="phecode_286-1": # Bipolar disorder
        covariates = [c for c in covariates if c not in ['bipolar_disorder']]
    if endpoint=="phecode_286-2": # Major depressive disorder
        covariates = [c for c in covariates if c not in ['major_depressive_disorder']]
    if endpoint=="phecode_287": # psychotic disorders
        covariates = [c for c in covariates if c not in ['schizophrenia']]
    if endpoint=="phecode_287-1": # schizophrenia
        covariates = [c for c in covariates if c not in ['schizophrenia']]
    if endpoint=="phecode_331": # headache
        covariates = [c for c in covariates if c!="migraine"]
    if endpoint=="phecode_331-6": # headache
        covariates = [c for c in covariates if c!="migraine"]
    if endpoint=="phecode_416": # atrial fibrillation
        covariates = [c for c in covariates if c not in ['atrial_fibrillation']]
    if endpoint=="phecode_416-2": # atrial fibrillation and flutter
        covariates = [c for c in covariates if c not in ['atrial_fibrillation']]
    if endpoint=="phecode_416-21": # atrial fibrillation
        covariates = [c for c in covariates if c not in ['atrial_fibrillation']]
    if endpoint=="phecode_584": # Renal failure
        covariates = [c for c in covariates if c not in ['renal_failure']]
    if endpoint=="phecode_605": # Male sexual dysfuction
        covariates = [c for c in covariates if c not in ['sex_Male', 'male_erectile_dysfunction']]
    if endpoint=="phecode_605-1": # Male sexual dysfuction
        covariates = [c for c in covariates if c not in ['sex_Male', 'male_erectile_dysfunction']]
    if endpoint=="phecode_700": # Diffuse diseases of connective tissue
        covariates = [c for c in covariates if c not in ['systemic_lupus_erythematosus']]
    if endpoint=="phecode_700-1": # Lupus
        covariates = [c for c in covariates if c not in ['systemic_lupus_erythematosus']]
    if endpoint=="phecode_700-11": # Systemic lupus erythematosus [SLE]	
        covariates = [c for c in covariates if c not in ['systemic_lupus_erythematosus']]
    if endpoint=="phecode_705": # Rheumatoid arthritis and other inflammatory
        covariates = [c for c in covariates if c not in ['rheumatoid_arthritis']]
    if endpoint=="phecode_705-1": # Rheumatoid arthritis and other inflammatory
        covariates = [c for c in covariates if c not in ['rheumatoid_arthritis']]
    # added by lukas
    if endpoint=='phecode_620':
        covariates = [c for c in covariates if c not in ['sex_Male', 'male_erectile_dysfunction']]
    if endpoint=='phecode_627':
        covariates = [c for c in covariates if c not in ['sex_Male', 'male_erectile_dysfunction']]
    if endpoint=='phecode_627-4':
        covariates = [c for c in covariates if c not in ['sex_Male', 'male_erectile_dysfunction']]
    return covariates

In [62]:
# problematic endpoints
problem_endpoints = [
    'phecode_620',
    'phecode_627',
    'phecode_627-4',
    #"phecode_181",
    #"phecode_202",
    #"phecode_202-1",
    #"phecode_286",
    #"phecode_287-1",
    #"phecode_331",
    #"phecode_416",
    #"phecode_416-2",
    #"phecode_416-21",
    #"phecode_584", # 
    #"phecode_605",
    #"phecode_700",
    #"phecode_700-1",
    #"phecode_700-11",
    #"phecode_705",
    #"phecode_705-1"
]

endpoints_md.query("endpoint==@problem_endpoints")

,Unnamed: 0,endpoint,eligable,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf
885,885,phecode_620,31756,160,0.005038,620.0,Dysplasia of female genital organs,Genitourinary,Female,0.0,620.0,0.0
903,903,phecode_627,23317,4428,0.189904,627.0,Menopausal and postmenopausal disorders,Genitourinary,Female,0.0,627.0,0.0
907,907,phecode_627-4,28188,1028,0.036469,627.4,Menorrhagia/Excessive and frequent menstruation,Genitourinary,Female,0.0,627.0,1.0


In [69]:
from lifelines import CoxPHFitter
endpoint = problem_endpoints[0]
partition = 20
feature_set = "QRISK3" # QRISK3+Retina
display(endpoints_md.query("endpoint==@endpoint"))

test_data = load_pickle(f"/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas_220603_fullrun/data/220603_fullrun/coxph/errordata_{endpoint}_{feature_set}_{partition}.p")
display(test_data.T)

features = get_features(endpoint, score_defs)
print(features)
covariates = features["ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.3"][feature_set]

# # clean covariates for the coxphs to fit
covariates = clean_covariates(endpoint, covariates)
print(covariates)

data_endpoint = test_data[covariates + [f"{endpoint}_event", f"{endpoint}_time"]].astype(np.float32)

cph = fit_cox(data_endpoint,#.drop(columns=["systemic_lupus_erythematosus"]), 
              feature_set, covariates, endpoint, penalizer=0, step_size=0.1)
cph.print_summary()

,Unnamed: 0,endpoint,eligable,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf
885,885,phecode_620,31756,160,0.005038,620.0,Dysplasia of female genital organs,Genitourinary,Female,0.0,620.0,0.0


eid,1000212,1000270,1000475,1000716,1000747,1001004,1001101,1001218,1001284,1001381,...,5250156,5250206,5250285,5250491,5250686,5250922,5251073,5251220,5251563,5251840
age,0.893126,1.382681,-0.575537,-1.799423,-0.453148,-1.065091,0.281183,-0.697925,1.382681,0.770738,...,-0.453148,0.036406,-1.799423,0.893126,-0.697925,1.260292,-0.453148,1.015515,-0.208371,1.137903
ethnic_background_Asian,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ethnic_background_Chinese,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ethnic_background_Black,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ethnic_background_Mixed,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
smoking_status_Previous,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
smoking_status_Current,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
bmi,-0.804325,-0.024947,0.002921,-0.786913,-0.153388,-0.979316,0.101206,-0.784024,-0.176420,-0.768110,...,-1.039608,1.993108,-0.953877,2.027573,-0.428950,-1.321041,-0.705492,-0.804325,-1.080327,0.697968
diabetes1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
diabetes2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


{'ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.3': {'QRISK3': ['age', 'sex_Male', 'ethnic_background_Asian', 'ethnic_background_Chinese', 'ethnic_background_Black', 'ethnic_background_Mixed', 'smoking_status_Previous', 'smoking_status_Current', 'bmi', 'diabetes1', 'diabetes2', 'fh_heart_disease', 'renal_failure', 'atrial_fibrillation', 'migraine', 'rheumatoid_arthritis', 'systemic_lupus_erythematosus', 'schizophrenia', 'bipolar_disorder', 'major_depressive_disorder', 'male_erectile_dysfunction', 'antihypertensives', 'corticosteroids', 'psycholeptics', 'systolic_blood_pressure', 'cholesterol', 'hdl_cholesterol'], 'QRISK3+Retina': ['age', 'sex_Male', 'ethnic_background_Asian', 'ethnic_background_Chinese', 'ethnic_background_Black', 'ethnic_background_Mixed', 'smoking_status_Previous', 'smoking_status_Current', 'bmi', 'diabetes1', 'diabetes2', 'fh_heart_disease', 'renal_failure', 'atrial_fibrillation', 'migraine', 'rheumatoid_arthritis', 'systemic_lupus_erythematosus', 'schizo

/home/loockl/miniconda3/envs/retina_risk/lib/python3.9/site-packages/lifelines/utils/__init__.py:1123: ConvergenceWarning: Column ethnic_background_Chinese have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phecode_620_event'].astype(bool)
>>> print(df.loc[events, 'ethnic_background_Chinese'].var())
>>> print(df.loc[~events, 'ethnic_background_Chinese'].var())

A very low variance means that the column ethnic_background_Chinese completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/home/loockl/miniconda3/envs/retina_risk/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1594: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta)

<lifelines.CoxPHFitter: fitted with 26854 total observations, 26719 right-censored observations>
             duration col = 'phecode_620_time'
                event col = 'phecode_620_event'
      baseline estimation = breslow
   number of observations = 26854
number of events observed = 135
   partial log-likelihood = -1318.27
         time fit was run = 2022-07-01 13:25:43 UTC

---
                               coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                            
age                           -0.73       0.48       0.10            -0.94            -0.53                 0.39                 0.59
ethnic_background_Asian        0.34       1.40       0.47            -0.58             1.25                 0.56                 3.48
ethnic_background_Chinese    -15.66       0.00    2944.21         -5786.21          5754.89                 0.00                  inf
ethnic_background_Black        0.79       2.20       0.34             0.12             1.46                 1.13                 4.31
ethnic_background_Mixed       -0.68       0.51       1.01            -2.65             1.29                 0.07                 3.65
smoking_status_Previous        0.48       1.62       0.20             0.10             0.87                 1.10                 2.39
smoking_status_Current         1.00       2.72       0.24             0.53             1.48                 1.70                 4.37
bmi                           -0.16       0.85       0.10            -0.35             0.03                 0.70                 1.03
diabetes1                    -16.21       0.00    2975.69         -5848.46          5816.03                 0.00                  inf
diabetes2                      0.47       1.61       0.52            -0.55             1.50                 0.58                 4.49
fh_heart_disease               0.01       1.01       0.19            -0.36             0.37                 0.70                 1.45
renal_failure                -15.16       0.00    3548.60         -6970.28          6939.96                 0.00                  inf
atrial_fibrillation            0.89       2.44       1.01            -1.09             2.87                 0.34                17.65
migraine                       0.21       1.24       0.28            -0.33             0.75                 0.72                 2.13
rheumatoid_arthritis           0.47       1.59       0.72            -0.94             1.87                 0.39                 6.50
systemic_lupus_erythematosus   1.22       3.39       1.01            -0.76             3.20                 0.47                24.56
schizophrenia                -16.22       0.00    5019.31         -9853.89          9821.45                 0.00                  inf
bipolar_disorder               0.09       1.09       1.04            -1.95             2.12                 0.14                 8.31
major_depressive_disorder      0.20       1.22       0.21            -0.21             0.61                 0.81                 1.85
antihypertensives            -15.15       0.00    2180.20         -4288.26          4257.95                 0.00                  inf
corticosteroids              -15.67       0.00    5932.65        -11643.46         11612.11                 0.00                  inf
psycholeptics                  0.76       2.13       0.48            -0.18             1.69                 0.83                 5.42
systolic_blood_pressure        0.02       1.02       0.10            -0.18             0.21                 0.84                 1.23
cholesterol                    0.05       1.05       0.10            -0.15             0.24                 0.86                 1.28
hdl_cholesterol                0.20       1.22       0.09             0.01             0.38                 1.01                

<lifelines.CoxPHFitter: fitted with 396241 total observations, 384393 right-censored observations>
             duration col = 'phecode_181_time'
                event col = 'phecode_181_event'
      baseline estimation = breslow
   number of observations = 396241
number of events observed = 11848
   partial log-likelihood = -150483.48
         time fit was run = 2022-06-14 10:22:14 UTC

---
                            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                         
age                         0.20       1.22       0.01             0.18             0.22                 1.19                 1.24
sex_Male                   -0.48       0.62       0.02            -0.52            -0.44                 0.59                 0.65
ethnic_background_Asian     0.24       1.27       0.06             0.12             0.36                 1.12                 1.43
ethnic_background_Chinese  -0.51       0.60       0.23            -0.96            -0.06                 0.38                 0.94
ethnic_background_Black     0.04       1.04       0.07            -0.11             0.18                 0.90                 1.20
ethnic_background_Mixed     0.03       1.03       0.12            -0.21             0.26                 0.81                 1.30
smoking_status_Previous     0.25       1.28       0.02             0.21             0.29                 1.23                 1.34
smoking_status_Current      0.43       1.54       0.03             0.38             0.49                 1.46                 1.63
bmi                         0.06       1.06       0.01             0.04             0.08                 1.04                 1.08
diabetes1                   0.65       1.92       0.09             0.48             0.82                 1.62                 2.28
diabetes2                  -0.02       0.98       0.04            -0.11             0.06                 0.90                 1.07
fh_heart_disease            0.03       1.03       0.02            -0.01             0.07                 0.99                 1.07
renal_failure               0.23       1.26       0.12            -0.00             0.46                 1.00                 1.59
atrial_fibrillation         0.04       1.04       0.08            -0.13             0.20                 0.88                 1.23
migraine                    0.24       1.27       0.03             0.17             0.31                 1.19                 1.36
rheumatoid_arthritis        2.38      10.80       0.32             1.76             3.00                 5.81                20.07
schizophrenia              -0.03       0.97       0.19            -0.40             0.34                 0.67                 1.40
bipolar_disorder           -0.34       0.71       0.18            -0.69            -0.00                 0.50                 1.00
major_depressive_disorder   0.24       1.27       0.02             0.19             0.28                 1.21                 1.33
male_erectile_dysfunction   0.01       1.01       0.05            -0.09             0.11                 0.92                 1.11
antihypertensives           0.20       1.22       0.06             0.07             0.32                 1.08                 1.38
corticosteroids             1.37       3.94       0.16             1.05             1.69                 2.87                 5.43
psycholeptics               0.29       1.34       0.06             0.17             0.41                 1.19                 1.50
systolic_blood_pressure    -0.03       0.97       0.01            -0.05            -0.01                 0.96                 0.99
cholesterol                -0.10       0.91       0.01            -0.12            -0.08                 0.89                 0.93
hdl_cholesterol            -0.13       0.88       0.01            -0

In [48]:
jobids

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
#fit_partition(in_path, model_path, score_defs, 0)

In [8]:
import submitit

partitions = [i for i in range(22)]

executor = submitit.AutoExecutor(folder="log_test/%j")
# set timeout in min, and partition for running the job
executor.update_parameters(slurm_array_parallelism=6,
                           nodes=1,
                           #slurm_mem="500G",
                           timeout_min=600,
                          #slurm_setup=[
                          #  """export MKL_NUM_THREADS=1""",
                          #  """export NUMEXPR_NUM_THREADS=1""",
                           # """export OMP_NUM_THREADS=1"""]
                          )

job = executor.map_array(fit_partition, partitions)  


# jobs = []
# for partition in tqdm(partitions):
#     job = executor.submit(fit_partition, in_path, model_path, score_defs, partition)
#     jobs.append(job)

TypeError: cannot pickle '_hashlib.HASH' object

In [9]:
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

import ray
#ray start --head --port=6379 --num-cpus 64 # in terminal
#ray.init(address='auto')#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))
#ray.init(num_cpus=32)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.7', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '10.32.105.2', 'raylet_ip_address': '10.32.105.2', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-13_11-43-47_176488_2125616/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-13_11-43-47_176488_2125616/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-06-13_11-43-47_176488_2125616', 'metrics_export_port': 63797, 'gcs_address': '10.32.105.2:6379', 'address': '10.32.105.2:6379', 'node_id': 'ebe5c1cc59f8f70c07421a2fbef0a2c0a0f9550a21554a5a287c320f'})

In [10]:
ray.available_resources()

{'node:10.32.105.2': 1.0,
 'object_store_memory': 158159606169.0,
 'memory': 359039081063.0,
 'CPU': 64.0}

In [ ]:
for partition in tqdm(partitions):
    fit_partition()

  0%|          | 0/22 [00:00<?, ?it/s]

Ray object not yet initialised


  0%|          | 0/1883 [00:00<?, ?it/s]

(raylet) [2022-06-13 11:45:42,810 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2143681) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 11:45:42,815 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2143687) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 11:45:42,820 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2143694) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 11:45:42,825 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2143711) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 11:45:42,830 E 2125963 2125963] (raylet) worker_pool.cc:518

(fit_endpoint pid=2143451) ConvergenceError Identity(Records)+MLP phecode_084-2 ASCVD 0 problem: reduce step size


...

(fit_endpoint pid=2143823) /home/steinfej/miniconda3/envs/ehrgraphs/lib/python3.9/site-packages/lifelines/utils/__init__.py:1123: ConvergenceWarning: Column ethnic_background_Chinese have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:
(fit_endpoint pid=2143823) 
(fit_endpoint pid=2143823) >>> events = df['phecode_387-5_event'].astype(bool)
(fit_endpoint pid=2143823) >>> print(df.loc[events, 'ethnic_background_Chinese'].var())
(fit_endpoint pid=2143823) >>> print(df.loc[~events, 'ethnic_background_Chinese'].var())
(fit_endpoint pid=2143823) 
(fit_endpoint pid=2143823) A very low variance means that the column ethnic_background_Chinese completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.
(fit_endpoint pid=2143823) 
(fit_endpoint pid=2143823)   warnin

(fit_endpoint pid=2143827) ConvergenceError Identity(Records)+MLP phecode_390-5 QRISK3 0 problem: reduce step size


(fit_endpoint pid=2143827) /home/steinfej/miniconda3/envs/ehrgraphs/lib/python3.9/site-packages/lifelines/utils/__init__.py:1123: ConvergenceWarning: Column ethnic_background_Chinese have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:
(fit_endpoint pid=2143827) 
(fit_endpoint pid=2143827) >>> events = df['phecode_390-5_event'].astype(bool)
(fit_endpoint pid=2143827) >>> print(df.loc[events, 'ethnic_background_Chinese'].var())
(fit_endpoint pid=2143827) >>> print(df.loc[~events, 'ethnic_background_Chinese'].var())
(fit_endpoint pid=2143827) 
(fit_endpoint pid=2143827) A very low variance means that the column ethnic_background_Chinese completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.
(fit_endpoint pid=2143827) 
(fit_endpoint pid=2143827)   warnin

(fit_endpoint pid=2143827) ConvergenceError Identity(Records)+MLP phecode_390-5 QRISK3 0 trying with reduced step size ... 0.5 successfull


(raylet) [2022-06-13 12:25:16,490 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2160067) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 12:25:16,495 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2160068) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 12:25:16,499 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2160069) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 12:25:16,504 E 2125963 2125963] (raylet) worker_pool.cc:518: Some workers of the worker process(2160070) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-13 12:25:16,508 E 2125963 2125963] (raylet) worker_pool.cc:518

In [ ]:
## Debugging

In [43]:
# fit_endpoint(data_partition, eids_dict, endpoint_defs, endpoints[0], partition, models, model_path)

True

In [44]:
cph_1 = load_pickle("/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220613/coxph/models/OMOP_4306655_QRISK3_0.p")
cph_2 = load_pickle("/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220613/coxph/models/OMOP_4306655_Age+Sex+MedicalHistory_0.p")

In [45]:
cph_1.print_summary()

<lifelines.CoxPHFitter: fitted with 389520 total observations, 360325 right-censored observations>
             duration col = 'OMOP_4306655_time'
                event col = 'OMOP_4306655_event'
      baseline estimation = breslow
   number of observations = 389520
number of events observed = 29195
   partial log-likelihood = -358964.51
         time fit was run = 2022-06-13 08:51:04 UTC

---
                               coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                            
age                            0.80       2.22       0.01             0.78             0.81                 2.19                 2.26
sex_Male                       0.35       1.42       0.01             0.32             0.38                 1.38                 1.46
ethnic_background_Asian       -0.16       0.85       0.05            -0.26            -0.06                 0.77                 0.94
ethnic_background_Chinese     -0.13       0.88       0.15            -0.41             0.16                 0.66                 1.17
ethnic_background_Black       -0.08       0.92       0.06            -0.19             0.03                 0.82                 1.03
ethnic_background_Mixed       -0.13       0.88       0.10            -0.32             0.06                 0.73                 1.06
smoking_status_Previous        0.29       1.33       0.01             0.26             0.31                 1.30                 1.37
smoking_status_Current         1.04       2.84       0.02             1.01             1.08                 2.75                 2.94
bmi                            0.12       1.13       0.01             0.11             0.14                 1.12                 1.15
diabetes1                      0.86       2.36       0.04             0.77             0.94                 2.16                 2.57
diabetes2                      0.12       1.13       0.02             0.08             0.17                 1.08                 1.19
fh_heart_disease              -0.01       0.99       0.01            -0.03             0.02                 0.97                 1.02
renal_failure                  0.81       2.25       0.05             0.71             0.91                 2.04                 2.47
atrial_fibrillation           -0.10       0.91       0.05            -0.19            -0.00                 0.83                 1.00
migraine                      -0.66       0.52       0.04            -0.73            -0.59                 0.48                 0.56
rheumatoid_arthritis           0.43       1.54       0.05             0.34             0.52                 1.41                 1.68
systemic_lupus_erythematosus   0.60       1.83       0.14             0.34             0.87                 1.40                 2.38
schizophrenia                  0.19       1.21       0.10             0.00             0.38                 1.00                 1.46
bipolar_disorder               0.35       1.42       0.09             0.18             0.51                 1.20                 1.67
major_depressive_disorder     -0.37       0.69       0.02            -0.41            -0.33                 0.66                 0.72
male_erectile_dysfunction     -0.58       0.56       0.03            -0.63            -0.52                 0.53                 0.60
antihypertensives              0.25       1.29       0.03             0.19             0.32                 1.21                 1.37
corticosteroids                1.00       2.72       0.10             0.80             1.20                 2.23                 3.34
psycholeptics                  0.65       1.92       0.03             0.59             0.71                 1.80                 2.04
systolic_blood_pressure        0.02       1.02       0.01             0.01             0.03                 1.01       

In [46]:
cph_2.print_summary()

<lifelines.CoxPHFitter: fitted with 389520 total observations, 360325 right-censored observations>
             duration col = 'OMOP_4306655_time'
                event col = 'OMOP_4306655_event'
      baseline estimation = breslow
   number of observations = 389520
number of events observed = 29195
   partial log-likelihood = -333358.67
         time fit was run = 2022-06-13 08:51:18 UTC

---
               coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                            
age            0.41       1.51       0.01             0.40             0.43                 1.49                 1.53
sex_Male       0.13       1.14       0.01             0.11             0.16                 1.12                 1.17
OMOP_4306655   1.01       2.74       0.00             1.00             1.02                 2.72                 2.77

               cmp to      z      p   -log2(p)
covariate                                     
age              0.00  55.05 <0.005        inf
sex_Male         0.00  11.14 <0.005      93.31
OMOP_4306655     0.00 237.90 <0.005        inf
---
Concordance = 0.89
Partial AIC = 666723.34
log-likelihood ratio test = 75163.39 on 3 df
-log2(p) of ll-ratio test = inf

In [ ]:
encode_cols = 
pd.get_dummies(data_partition['Identity(Records)+MLP'], columns=["ethnic_background", "sex", "smoking_status"], prefix=)

In [38]:
data_partition['Identity(Records)+MLP'].select_dtypes("category")

,partition,split,model,ethnic_background,sex,smoking_status
eid,,,,,,
1000018,0,train,Identity(Records)+MLP,White,Female,Current
1000020,0,train,Identity(Records)+MLP,White,Male,Current
1000037,0,train,Identity(Records)+MLP,White,Female,Previous
1000043,0,train,Identity(Records)+MLP,White,Male,Previous
1000051,0,train,Identity(Records)+MLP,White,Female,Never
...,...,...,...,...,...,...
5337676,0,train,Identity(Records)+MLP,White,Male,Never
5337681,0,train,Identity(Records)+MLP,White,Male,Previous
5337697,0,train,Identity(Records)+MLP,White,Male,Previous


(raylet) [2022-06-12 21:30:41,739 E 2586297 2586297] (raylet) worker_pool.cc:518: Some workers of the worker process(2954666) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-12 21:30:41,743 E 2586297 2586297] (raylet) worker_pool.cc:518: Some workers of the worker process(2954667) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-12 21:30:41,747 E 2586297 2586297] (raylet) worker_pool.cc:518: Some workers of the worker process(2954668) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-12 21:30:41,752 E 2586297 2586297] (raylet) worker_pool.cc:518: Some workers of the worker process(2954669) have not registered within the timeout. The process is still alive, probably it's hanging during start.
(raylet) [2022-06-12 21:30:41,756 E 2586297 2586297] (raylet) worker_pool.cc:518

In [32]:
load_pickle("/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/coxph/errordata_phecode_002-1_Age+Sex+MedicalHistory+I(Age*MH)_0.p")

,age_at_recruitment_f21022_0_0,sex_f31_0_0,phecode_002-1,phecode_002-1_event,phecode_002-1_time
eid,,,,,
1000018,-0.931072,0.0,0.086565,0.0,11.866089
1000020,0.304493,1.0,-1.797872,0.0,13.596446
1000037,0.304493,0.0,0.259373,0.0,12.868163
1000043,0.798719,1.0,2.246452,0.0,12.309629
1000051,-0.683959,0.0,0.685702,0.0,15.291210
...,...,...,...,...,...
5337676,0.922275,1.0,0.725549,0.0,11.518374
5337681,1.292945,1.0,0.346841,0.0,12.312367
5337697,-1.919524,1.0,0.265374,0.0,11.822283


In [22]:
data_partition['Identity(Records)+MLP']['phecode_977']

eid
1303905    0.507814
1303918    0.033095
1303920   -0.946619
1303937    0.042184
1303943   -0.270619
             ...   
5316968   -0.614777
5316970    0.009382
5316985   -0.361461
5316994   -0.194883
5317002   -0.581893
Name: phecode_977, Length: 401263, dtype: float64

# old stuff

In [4]:
!ls -al {output_path}

total 26289984
drwxrwx--- 3 loockl posix-nogroup         378 Jun  8 12:38 .
drwxrwx--- 5 loockl posix-nogroup          77 Jun  8 10:15 ..
drwxrwx--- 4 loockl posix-nogroup         118 Jun  8 13:06 2af9tvdp
-rwxrwx--- 1 loockl posix-nogroup  4913114786 Jun  8 11:54 baseline_outcomes_220531.feather
-rwxrwx--- 1 loockl posix-nogroup 10680216930 Jun  8 11:55 baseline_outcomes_long_220531.feather
-rwxrwx--- 1 loockl posix-nogroup  2247323418 Jun  8 12:35 eligable_eids_2022-06-08.feather
-rwxrwx--- 1 loockl posix-nogroup  5462517522 Jun  8 11:56 eligable_eids_220531.feather
-rwxrwx--- 1 loockl posix-nogroup  2233384746 Jun  8 12:38 eligable_eids_long_2022-06-08.feather
-rwxrwx--- 1 loockl posix-nogroup  5425959242 Jun  8 11:56 eligable_eids_long_220531.feather
-rwxrwx--- 1 loockl posix-nogroup      195482 Jun  8 11:56 phecode_defs_220306.feather


In [5]:
print(output_path)
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_220531.feather").set_index("eid")
data_outcomes

/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas_220608/data


,OMOP_4306655_prev,phecode_202_prev,phecode_401_prev,phecode_401-1_prev,phecode_475_prev,phecode_202-2_prev,phecode_718_prev,phecode_089_prev,phecode_130_prev,phecode_713_prev,...,phecode_247-85_time,phecode_361-24_time,phecode_424-7_time,phecode_244-3_time,phecode_824-1_time,phecode_206-1_time,phecode_018_time,phecode_363-8_time,phecode_396-6_time,phecode_361-93_time
eid,,,,,,,,,,,,,,,,,,,,,
1000018,False,False,True,True,False,False,False,False,False,False,...,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089,11.866089
1000020,False,False,False,False,False,False,False,False,False,False,...,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446,13.596446
1000037,False,False,False,False,False,False,True,False,False,True,...,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163,12.868163
1000043,False,False,True,True,False,False,False,True,True,True,...,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629,12.309629
1000051,False,True,False,False,False,True,False,False,False,False,...,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210,15.291210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025150,False,False,False,False,False,False,False,False,False,False,...,14.237117,14.237117,14.237117,14.237117,14.237117,14.237117,14.237117,14.237117,14.237117,14.237117
6025165,False,False,False,False,False,False,True,True,False,False,...,13.059816,13.059816,13.059816,13.059816,13.059816,13.059816,13.059816,13.059816,13.059816,13.059816
6025173,False,False,False,False,False,False,False,False,False,False,...,13.018748,13.018748,13.018748,13.018748,13.018748,13.018748,13.018748,13.018748,13.018748,13.018748


In [6]:
import pandas as pd
all_endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_retinal_risk/data/220602/endpoints.csv').endpoint.values)])

#all_endpoints = sorted(endpoints_all_md.endpoint.to_list())
print(len(all_endpoints))

endpoints_not_overlapping_with_preds = []
#endpoints_not_overlapping_with_preds_md = pd.read_csv(f"{experiment_path}/endpoints_not_overlapping.csv", header=None)
#print(len(endpoints_not_overlapping_with_preds_md))
#endpoints_not_overlapping_with_preds = list(endpoints_not_overlapping_with_preds_md[0].values)

endpoints = []
for c in all_endpoints:
    if c not in endpoints_not_overlapping_with_preds: # this is what i want
        #print('OK    - ',c)
        endpoints.append(c)
    #if c in endpoints_not_overlapping_with_preds: # this is what causes errors!
    #    print('ERROR - ',c)
print(len(endpoints))

1171
1171


In [7]:
splits = ["train", "valid", 'test'] # "test_left", 'test_right'

In [8]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").query("endpoint==@endpoints").sort_values("endpoint").set_index("endpoint")

In [9]:
from datetime import date
today = str(date.today())

In [10]:
eligable_eids = pd.read_feather(f"{output_path}/eligable_eids_{today}.feather") # TODO CHANGE!
eids_dict = eligable_eids.set_index("endpoint")["eid_list"].to_dict()

In [11]:
%env MKL_NUM_THREADS=4
%env NUMEXPR_NUM_THREADS=4
%env OMP_NUM_THREADS=4

env: MKL_NUM_THREADS=4
env: NUMEXPR_NUM_THREADS=4
env: OMP_NUM_THREADS=4


In [12]:
ray.shutdown()

In [13]:
import ray

ray.init(num_cpus=24)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

RayContext(dashboard_url='', python_version='3.9.7', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '10.32.105.2', 'raylet_ip_address': '10.32.105.2', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-08_13-08-03_469261_2248531/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-08_13-08-03_469261_2248531/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-06-08_13-08-03_469261_2248531', 'metrics_export_port': 63944, 'gcs_address': '10.32.105.2:55646', 'address': '10.32.105.2:55646', 'node_id': '532f4fbe25b670f472abbe1d901de698ad9ebfed3d5286753fe3bb87'})

In [14]:
AgeSex = ["age_at_recruitment_f21022_0_0", "sex_f31_0_0"]

# Train COX

In [15]:
in_path = pathlib.Path(f"{experiment_path}/coxph/input")
in_path.mkdir(parents=True, exist_ok=True)

model_path = f"{experiment_path}/coxph/models"
pathlib.Path(model_path).mkdir(parents=True, exist_ok=True)

In [16]:
models = [f.name for f in in_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]
models

['ImageTraining_[]_SimpleViT_MLPHead']

In [17]:
from formulaic.errors import FactorEvaluationError

In [18]:
in_path

PosixPath('/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas_220608/data/2af9tvdp/coxph/input')

In [19]:
from lifelines import CoxPHFitter
from lifelines.exceptions import ConvergenceError
import zstandard
import pickle

def get_features(endpoint):
    features = {
        models[0]: { # TODO CHANGE!
            "Age+Sex": AgeSex,
            "Retina": [endpoint],
            "Age+Sex+Retina": AgeSex + [endpoint],
            #"Age+Sex+MedicalHistory+I(Age*MH)": AgeSex + [endpoint]
            }
    }
    return features

def get_train_data(in_path, partition, models, mapping):
    train_data = {
        model: pd.read_feather(f"{in_path}/{model}/{partition}/train.feather").set_index("eid").merge(data_outcomes, left_index=True, right_index=True, how="left").replace(mapping)
    for model in models}
    
    return train_data

def fit_cox(data_fit, feature_set, covariates, endpoint, penalizer, step_size=1):
    if feature_set=="Age+Sex+MedicalHistory+I(Age*MH)":
        endpoint_label = endpoint.replace("-", "")
        data_fit.columns = [c.replace("-", "") for c in data_fit.columns]
        covariates = [c.replace("-", "") for c in covariates]
        #print(endpoint_label)
        #print(data_fit)
        #print(covariates)
        if "sex_f31_0_0" in covariates:
            formula=f"age_at_recruitment_f21022_0_0*{endpoint_label}+sex_f31_0_0*{endpoint_label}"
        else:
            formula=f"age_at_recruitment_f21022_0_0*{endpoint_label}"
        cph = CoxPHFitter(penalizer=penalizer)
        cph.fit(data_fit, f"{endpoint_label}_time", f"{endpoint_label}_event", formula=formula, step_size=step_size)
    else:
        cph = CoxPHFitter(penalizer=penalizer)
        cph.fit(data_fit, f"{endpoint}_time", f"{endpoint}_event", step_size=step_size)

    return cph

def save_pickle(data, data_path):
    with open(data_path, "wb") as fh:
        cctx = zstandard.ZstdCompressor()
        with cctx.stream_writer(fh) as compressor:
            compressor.write(pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL))
            
def load_pickle(fp):
    with open(fp, "rb") as fh:
        dctx = zstandard.ZstdDecompressor()
        with dctx.stream_reader(fh) as decompressor:
            data = pickle.loads(decompressor.read())
    return data

@ray.remote
def fit_endpoint(data_partition, eids_dict, endpoint_defs, endpoint, partition, models, model_path):
    eids_incl = eids_dict[endpoint].tolist()
    features = get_features(endpoint)
    eligibility = endpoint_defs.loc[endpoint]["sex"]
    for model in models:
        data_model = data_partition[model]
        for feature_set, covariates in features[model].items():
            cph_path = f"{model_path}/{endpoint}_{feature_set}_{partition}.p"
            if os.path.isfile(cph_path):
                try:
                    cph = load_pickle(cph_path)
                    success = True
                except:
                    success = False
                    pass
            if not os.path.isfile(cph_path) or success==False:
                if (eligibility != "Both") and ("sex_f31_0_0" in covariates): 
                    covariates = [c for c in covariates if c!="sex_f31_0_0"]
                #print('covariates:', covariates)
                data_endpoint = data_model[covariates + [f"{endpoint}_event", f"{endpoint}_time"]].astype(np.float32)
                data_endpoint = data_endpoint[data_endpoint.index.isin(eids_incl)]
                try:
                    cph = fit_cox(data_endpoint, feature_set, covariates, endpoint, penalizer=0.0)
                    save_pickle(cph, cph_path)
                except (ValueError, ConvergenceError, KeyError,FactorEvaluationError) as e:
                    print("ConvergenceError", model, endpoint, feature_set, partition, "problem: reduce step size")
                    try:
                        cph = fit_cox(data_endpoint, feature_set, covariates, endpoint, penalizer=0.0, step_size=0.5)
                        save_pickle(cph, cph_path)
                        print("ConvergenceError", model, endpoint, feature_set, partition, "trying with reduced step size ... 0.5 successfull")
                    except (ValueError, ConvergenceError, KeyError,FactorEvaluationError) as e:
                        print("ConvergenceError", model, endpoint, feature_set, partition, "trying with reduced step size ... 0.5 failed")
                        try:
                            cph = fit_cox(data_endpoint, feature_set, covariates, endpoint, penalizer=0.0, step_size=0.1)
                            save_pickle(cph, cph_path)
                            print("ConvergenceError", model, endpoint, feature_set, partition, "trying with reduced step size ... 0.1 successfull")
                        except (ValueError, ConvergenceError, KeyError, FactorEvaluationError) as e:
                            print("ConvergenceError", model, endpoint, feature_set, partition, "trying with reduced step size ... 0.1 failed")
                            save_pickle(data_endpoint, f"{experiment_path}/coxph/errordata_{endpoint}_{feature_set}_{partition}.p")
                            pass
    return True

In [20]:
f"{experiment_path}/coxph"

'/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas_220608/data/2af9tvdp/coxph'

In [21]:
model_list =  !ls $model_path
#model_list = [m for m in model_list if "I(" in m]
model_list = [m for m in model_list]

In [22]:
model_list

[]

In [22]:
1+1

2

In [23]:
mapping = {"sex_f31_0_0": {"Female":0, "Male":1}}

ray_eids = ray.put(eids_dict)
ray_endpoint_defs = ray.put(endpoint_defs)
for partition in tqdm([0]): # in tqdm(partitions) # TODO: CHANGE!
    try:
        del ray_partition
    except:
        print("Ray object not yet initialised")
    try:
        data_partition = get_train_data(in_path, partition, models, mapping)
        ray_partition = ray.put(data_partition)
        progress = []
        for endpoint in endpoints:
            progress.append(fit_endpoint.remote(ray_partition, ray_eids, ray_endpoint_defs, endpoint, partition, models, model_path))
        [ray.get(s) for s in tqdm(progress)]
    except FileNotFoundError:
        print('file not found')
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

Ray object not yet initialised


  0%|          | 0/1171 [00:00<?, ?it/s]

(fit_endpoint pid=2248992) /home/loockl/miniconda3/envs/retina_risk/lib/python3.9/site-packages/lifelines/utils/__init__.py:1123: ConvergenceWarning: Column sex_f31_0_0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:
(fit_endpoint pid=2248992) 
(fit_endpoint pid=2248992) >>> events = df['phecode_140_event'].astype(bool)
(fit_endpoint pid=2248992) >>> print(df.loc[events, 'sex_f31_0_0'].var())
(fit_endpoint pid=2248992) >>> print(df.loc[~events, 'sex_f31_0_0'].var())
(fit_endpoint pid=2248992) 
(fit_endpoint pid=2248992) A very low variance means that the column sex_f31_0_0 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.
(fit_endpoint pid=2248992) 
(fit_endpoint pid=2248992)   warnings.warn(dedent(warning_text), ConvergenceWarning)
(fit_end

In [33]:
load_pickle("/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas/data/test_experiment/coxph/models/phecode_841_Retina_0.p")

<lifelines.CoxPHFitter: fitted with 47954 total observations, 46466 right-censored observations>

In [22]:
data_partition['Identity(Records)+MLP']['phecode_977']

eid
1303905    0.507814
1303918    0.033095
1303920   -0.946619
1303937    0.042184
1303943   -0.270619
             ...   
5316968   -0.614777
5316970    0.009382
5316985   -0.361461
5316994   -0.194883
5317002   -0.581893
Name: phecode_977, Length: 401263, dtype: float64